<a href="https://colab.research.google.com/github/piotr-malek/my_python_projects/blob/main/strava_activities.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Hi! I combined my passion for sport and my goal to master the requests module into a practical task. I fetched data from Strava following the OAuth flow, transformed it using pandas, and sent over to Google Sheets. Then, I connected Google Sheets to Looker Studio and [built a personal dashboard](https://drive.google.com/file/d/1gceII7nffpeG8WjempazxkAYdNWEzwnb/view?usp=sharing) that updates with every run of a function.

In [9]:
import requests
import json
import os
from google.colab import userdata

try:
  import requests_cache
except ImportError:
  !pip install requests_cache
  import requests_cache

requests_cache.install_cache('strava_activities', expire_after=86400)

# Google Colab API only allows for fetching tokens but not updating them programmatically.
# Thus saving them right away as environment variables
if 'client_id' not in os.environ:
    os.environ['client_id'] = userdata.get('client_id')
if 'client_secret' not in os.environ:
    os.environ['client_secret'] = userdata.get('client_secret')
if 'refresh_token' not in os.environ:
    os.environ['refresh_token'] = userdata.get('refresh_token')
if 'access_token' not in os.environ:
    os.environ['access_token'] = userdata.get('access_token')

# Strava API resets access tokens every 6 hours, thus a function to refresh tokens
def refresh_access_token():
    client_id = os.getenv('client_id')
    client_secret = os.getenv('client_secret')
    refresh_token = os.getenv('refresh_token')

    payload = {
        'client_id': client_id,
        'client_secret': client_secret,
        'grant_type': 'refresh_token',
        'refresh_token': refresh_token
    }

    url = 'https://www.strava.com/oauth/token'

    response = requests.post(url, data=payload)
    if response.status_code == 200:
        tokens = response.json()
        new_access_token = tokens['access_token']
        new_refresh_token = tokens['refresh_token']

        os.environ['access_token'] = new_access_token
        os.environ['refresh_token'] = new_refresh_token

        return new_access_token
    else:
        print(f"Couldn't refresh tokens. The status code was: {response.status_code}")
        return None

def fetch_activities():
    access_token = os.getenv('access_token')

    url = 'https://www.strava.com/api/v3/athlete/activities'
    headers = {'Authorization': f'Bearer {access_token}'}
    params = {'per_page': 200}

    page = 1
    all_data = []
    # Mechamism to only retry once if token fails
    token_refreshed = False

    while True:
        params['page'] = page
        response = requests.get(url, headers=headers, params=params)

        if response.status_code == 401 and not token_refreshed:
            # If auth error refresh the token
            new_access_token = refresh_access_token()
            if new_access_token:
                headers['Authorization'] = f'Bearer {new_access_token}'
                token_refreshed = True
                continue  # Retry the request with the new token
            else:
                print("Failed to refresh the access token. Exiting.")
                break

        elif response.status_code != 200:
            print(f"Request failed, error code {response.status_code}")
            break

        data = response.json()

        if not data:
            break

        all_data.extend(data)
        page += 1

    return all_data

activities = fetch_activities()
# print the latest activity to get an idea of what the response looks like or return an error
if activities:
    print(json.dumps(activities[0], indent=4))
else:
    print("No activities fetched.")


{
    "resource_state": 2,
    "athlete": {
        "id": 15953995,
        "resource_state": 1
    },
    "name": "Morning Ride",
    "distance": 5622.8,
    "moving_time": 1267,
    "elapsed_time": 1535,
    "total_elevation_gain": 62.0,
    "type": "Ride",
    "sport_type": "Ride",
    "workout_type": 10,
    "id": 11593620909,
    "start_date": "2024-06-07T06:56:23Z",
    "start_date_local": "2024-06-07T08:56:23Z",
    "timezone": "(GMT+01:00) Europe/Warsaw",
    "utc_offset": 7200.0,
    "location_city": null,
    "location_state": null,
    "location_country": "Poland",
    "achievement_count": 1,
    "kudos_count": 0,
    "comment_count": 0,
    "athlete_count": 1,
    "photo_count": 0,
    "map": {
        "id": "a11593620909",
        "summary_polyline": "edcpHcujyBZNBJADe@^]Ng@ZKNAF@^Nf@Zf@BBx@e@`@DJJNtA`@nBJPJDb@iBVu@LM`@QfAUv@[rA]dAi@r@WzAu@fBoA\\Q|B_BTYPKLY@WAa@Bs@j@uCDk@`@sBBk@h@_DDg@hA}Hb@gBvBaG^sBb@gDDq@?kAD_AAgBLaCL_@VsAL_@T[nAeCLgAGcA?o@PIx@KZo@l@STGvAIlC]~Ak@tAm@|@W\

In [10]:
import pandas as pd

df = []

# Fetch only the needed fields and rename them on the spot
for dct in activities:
  subset = {
        'name': dct.get('name', None),
        'distance': dct.get('distance', None),
        'elapsed_time': dct.get('elapsed_time', None),
        'moving_time': dct.get('moving_time', None),
        'total_elevation_gain': dct.get('total_elevation_gain', None),
        'type': dct.get('type', None),
        'start_date': dct.get('start_date_local', None),
        'achievements': dct.get('achievement_count', None),
        'kudoses': dct.get('kudos_count', None),
        'comments': dct.get('comment_count', None),
        'gear': dct.get('gear_id', None),
        'average_speed': dct.get('average_speed', None),
        'max_speed': dct.get('max_speed', None),
        'energy_output': dct.get('kilojoules', None),
        'average_heartrate': dct.get('average_heartrate', None),
        'max_heartrate': dct.get('max_heartrate', None),
        'elev_high': dct.get('elev_high', None),
        'elev_low': dct.get('elev_low', None),
        'personal_records': dct.get('pr_count', None),
        'relative_effort': dct.get('suffer_score', None)
    }
  df.append(subset)

df = pd.DataFrame(df)
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2347 entries, 0 to 2346
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   name                  2347 non-null   object 
 1   distance              2347 non-null   float64
 2   elapsed_time          2347 non-null   int64  
 3   moving_time           2347 non-null   int64  
 4   total_elevation_gain  2347 non-null   float64
 5   type                  2347 non-null   object 
 6   start_date            2347 non-null   object 
 7   achievements          2347 non-null   int64  
 8   kudoses               2347 non-null   int64  
 9   comments              2347 non-null   int64  
 10  gear                  2183 non-null   object 
 11  average_speed         2347 non-null   float64
 12  max_speed             2347 non-null   float64
 13  energy_output         1986 non-null   float64
 14  average_heartrate     791 non-null    float64
 15  max_heartrate        

In [11]:
df.head()

,name,distance,elapsed_time,moving_time,total_elevation_gain,type,start_date,achievements,kudoses,comments,gear,average_speed,max_speed,energy_output,average_heartrate,max_heartrate,elev_high,elev_low,personal_records,relative_effort
0,Morning Ride,5622.8,1535,1267,62.0,Ride,2024-06-07T08:56:23Z,1,0,0,b3015498,4.438,8.180,101.7,131.0,161.0,258.6,226.0,0,10.0
1,Lunch Ride,101423.0,14626,13663,655.0,Ride,2024-06-06T12:52:07Z,22,3,0,b6090713,7.423,16.174,1718.1,150.0,177.0,342.4,194.4,6,315.0
2,Morning Ride,5674.2,1834,1373,48.0,Ride,2024-06-06T08:46:47Z,0,0,0,b3015498,4.133,7.955,102.8,137.9,179.0,249.6,218.6,0,16.0
3,Morning Ride,6249.0,1803,1487,72.0,Ride,2024-06-05T08:45:54Z,0,0,0,b3015498,4.202,8.494,116.7,137.0,167.0,258.2,224.8,0,18.0
4,Morning Ride,47035.8,6121,5993,252.0,Ride,2024-06-03T09:03:17Z,27,3,0,b6090713,7.848,12.870,850.7,154.6,181.0,266.8,186.8,12,165.0


In [12]:
from datetime import datetime

# Create datetime objects and calculate new ones
df['start_date'] = pd.to_datetime(df['start_date'], format='%Y-%m-%dT%H:%M:%SZ')
df['elapsed_time'] = pd.to_timedelta(df['elapsed_time'], unit='s')
df['end_date'] = df['start_date'] + df['elapsed_time']

# Convert speed metrics to km/h
df['average_speed'] = round(df['average_speed'] * 3.6, 2)
df['max_speed'] = round(df['max_speed'] * 3.6, 2)

# Convert time metrics to hours
df['elapsed_time'] = round(df['elapsed_time'].dt.total_seconds() / 3600, 2)
df['moving_time'] = round(df['moving_time'] / 3600, 2)

# Convert distance to km
df['distance'] = round(df['distance'] / 1000, 2)

# Add id field to prevent Looker Studio from grouping activities with the same name
df = df.sort_values(by='start_date')
df['id'] = range(1, len(df) + 1)

In [13]:
df.groupby('gear')['distance'].sum()

gear
b3015498     16083.14
b6090713     10625.75
b6642066       227.97
g10806277      157.30
g16417316       21.31
g4711838       818.94
Name: distance, dtype: float64

In [14]:
# Group gear data into an identical format to what's displayed in Strava
# And use that information to convert gear into human-friendly names
gear_rebrand ={
   'b3015498': 'B-Twin Triban 520',
   'b6090713': 'Viner Settanta',
   'b6642066': 'Colnago CLX',
   'g10806277': 'Nike Pegasus 37',
   'g16417316': 'Barefoot',
   'g4711838': 'Nike Pegasus 35'}

df['gear'] = df['gear'].map(gear_rebrand)

In [15]:
df.head()

,name,distance,elapsed_time,moving_time,total_elevation_gain,type,start_date,achievements,kudoses,comments,...,max_speed,energy_output,average_heartrate,max_heartrate,elev_high,elev_low,personal_records,relative_effort,end_date,id
2346,Evening Ride,18.61,0.93,0.89,58.7,Ride,2016-06-28 18:01:24,11,0,0,...,51.84,NaN,NaN,NaN,234.7,197.4,0,NaN,2016-06-28 18:57:05,1
2345,Afternoon Ride,8.24,0.41,0.39,18.6,Ride,2016-06-29 15:53:21,0,0,0,...,48.60,148.8,NaN,NaN,234.5,200.3,0,NaN,2016-06-29 16:17:59,2
2344,Night Ride,9.12,0.41,0.39,47.6,Ride,2016-06-30 00:39:50,0,0,0,...,90.36,186.5,NaN,NaN,234.9,203.3,0,NaN,2016-06-30 01:04:29,3
2343,Afternoon Ride,7.09,0.31,0.30,0.0,Ride,2016-06-30 14:55:38,1,0,0,...,38.88,101.2,NaN,NaN,234.9,197.4,1,NaN,2016-06-30 15:14:13,4
2342,Evening Ride,21.65,1.05,0.88,59.3,Ride,2016-06-30 18:37:51,8,0,0,...,38.52,415.9,NaN,NaN,234.9,192.1,4,NaN,2016-06-30 19:40:59,5


In [16]:
from google.colab import auth
from google.auth import default
import pandas as pd
import gspread

# Convert Timestamp columns to string format to avoid GSheets errors
df['start_date'] = df['start_date'].astype(str)
df['end_date'] = df['end_date'].astype(str)

# Authenticate to send data to GSheets
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

worksheet = gc.open("strava_activities")
sheet1 = worksheet.sheet1

# Update the worksheet with the Strava data
sheet1.update([df.columns.values.tolist()] + df.fillna('').values.tolist())


{'spreadsheetId': '11aZWVcvQqDfpNTOsB_EIVwcfCUqv6iU3mSxAIEcvGPA',
 'updatedRange': 'Sheet1!A1:V2348',
 'updatedRows': 2348,
 'updatedColumns': 22,
 'updatedCells': 51656}

What's next?

I want to fully automate the data load to the dashboard. Colab offers a scheduling feature but I'm not willing to pay for it. Instead I'm planning to deploy this function on Google Cloud and take advantage of Cloud Scheduler.

Many thanks for reading!